2023/1/9<br>
This code is for make image patch of G biloba

In [6]:
import os
import cv2
import glob
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from skimage.filters import threshold_otsu, threshold_local

set path

In [2]:
#create directory
current_path=os.getcwd()
npz_path=os.path.join(current_path, "npz_file")
npz_im_path=os.path.join(npz_path, "image_registration")

save_path=os.path.join(current_path, "im_for_model")
train_im_path=os.path.join(save_path, "train_im")
train_gt_path=os.path.join(save_path, "train_gt")
val_im_path=os.path.join(save_path, "val_im")
val_gt_path=os.path.join(save_path, "val_gt")
test_im_path=os.path.join(save_path, "test_im")
test_ori_im_path=os.path.join(save_path, "test_ori_im")
test_angle_im_path=os.path.join(save_path, "test_angle_im")

path_list=[train_im_path, train_gt_path, val_im_path, val_gt_path, test_im_path, test_ori_im_path, test_angle_im_path]

for path in path_list:
    if os.path.exists(path)==False:
        os.makedirs(path)

define function

In [3]:
def create_save_patch(im, im_step, im_size, save_path, im_name):
    
    height_div=im.shape[0]//im_step
    width_div=im.shape[1]//im_step
    
    for i in range(height_div):
        for j in range(width_div):
            im_patch=im[i*im_step:i*im_step+im_size, j*im_step:j*im_step+im_size]

            if (i*im_step+im_size>im.shape[0]) or (j*im_step+im_size>im.shape[1]):
                pass
            else:
                if os.path.exists(save_path)==False:
                    os.makedirs(save_path)
                cv2.imwrite(os.path.join(save_path, str(im_name)+"_"+str(i).zfill(3)+"_"+str(j).zfill(3)+".png"), im_patch)
    
#set hyperparameter
im_step=256
im_size=512

create and save image patches

In [5]:
for curdir, _, files in os.walk(os.path.join(npz_im_path)):
    if len(files)>0:
        for file in files:
            if not "DS_Store" in file:
                
                #load data
                target_path=os.path.join(curdir, file)
                npz=np.load(target_path)
                MFA=npz["MFA"]
                FLM=npz["FLM"]
                angle=npz["angle"]
                
                if "sample04" in file:
                    #binarization
                    block_size = 151
                    local_thresh = threshold_local(FLM, block_size, offset=10)
                    FLM_binary = FLM > local_thresh

                    #omit noise
                    FLM_binary=np.uint8(FLM_binary*255)
                    bg=np.where(FLM==0)
                    FLM_binary[bg]=0
                    
                    #connected component
                    ret_val, labels, stats, centroids=cv2.connectedComponentsWithStats(FLM_binary)
                    FLM_binary=np.where(labels==1, 255, 0)
                    
                else:
                    #binarization
                    thres=threshold_otsu(FLM)
                    FLM_binary=np.uint8((FLM>thres)*255)

                #set lumen as constant
                lumen_ind=np.where(FLM_binary==0)
                MFA[lumen_ind]=0
                angle[lumen_ind]=-1

                #normalization
                MFA_norm=(MFA-np.min(MFA))/(np.max(MFA)-np.min(MFA))*255
                
                im_name=os.path.splitext(file)[0][:-2]
                create_save_patch(MFA_norm, im_step, im_size, os.path.join(test_im_path, im_name), im_name)
                create_save_patch(MFA, im_step, im_size, os.path.join(test_ori_im_path, im_name), im_name)
                create_save_patch(angle, im_step, im_size, os.path.join(test_angle_im_path, im_name), im_name)